In [51]:
as = [3.3 , 4.3] #prices public info
cs = [3.0,4.0] #costs, private info
n= 2
lag = 1
#info set is lagged 1 period back,

function init_Ws(as,cs,n,lag, W0)
    Ws = Dict()
    argumento=fill(as, n*lag + 1 )
    cartesian_as=collect(Iterators.product(argumento...))
    
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    for c=cartesian_cs
        for a=cartesian_as
                Ws["$(a),$(c)"] = (rand(W0),0)
        end
    end
    return Ws
end

W0 = init_Ws(as,cs,n,lag,[10,11,12])

Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (10, 0)
  "

In [52]:
function update_m(as,cs,n,lag,Ws)
    
    argumento=fill(as, n*lag)
    cartesian_as=collect(Iterators.product(argumento...))
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    
    ms = Dict()
    for other_a=cartesian_as
        for c=cartesian_cs
            for my_a=as
                #println("my_a ", my_a)
                #check if key is there initilize to my_a
                if ! haskey(ms, "$(other_a),$(c)") 
                    ms["$(other_a),$(c)"] = my_a
                end
                
                current_pol = ms["$(other_a),$(c)"]
                current_state = Tuple(append!([current_pol],other_a ))
                other_state = Tuple(append!([my_a],other_a))
                
                #if already there set to my a if greater
                #println(my_a, ", ", current_state,", " ,other_state, ", ", Ws["$(current_state),$(c)"], ", ",  Ws["$(other_state),$(c)"] )
                if Ws["$(current_state),$(c)"][1] <= Ws["$(other_state),$(c)"][1]
                    ms["$(other_a),$(c)"] = my_a
                end
            end
        end
    end
    m(as,omegas) = ms["$(Tuple(as)),$(Tuple(omegas))"]
    return m
end


m0 = update_m(as,cs,n,lag,W0)
print(m0)
m0((4.3, 4.3),(3.,3.))

getfield(Main, Symbol("#m#7")){Dict{Any,Any}}(Dict{Any,Any}("(3.3, 3.3),(3.0, 4.0)"=>4.3,"(4.3, 4.3),(4.0, 4.0)"=>4.3,"(3.3, 4.3),(3.0, 3.0)"=>3.3,"(4.3, 4.3),(3.0, 3.0)"=>4.3,"(3.3, 3.3),(3.0, 3.0)"=>3.3,"(3.3, 3.3),(4.0, 3.0)"=>4.3,"(4.3, 4.3),(3.0, 4.0)"=>4.3,"(4.3, 3.3),(4.0, 3.0)"=>3.3,"(3.3, 4.3),(3.0, 4.0)"=>3.3,"(3.3, 4.3),(4.0, 3.0)"=>4.3,"(3.3, 4.3),(4.0, 4.0)"=>4.3,"(3.3, 3.3),(4.0, 4.0)"=>4.3,"(4.3, 4.3),(4.0, 3.0)"=>4.3,"(4.3, 3.3),(3.0, 4.0)"=>4.3,"(4.3, 3.3),(4.0, 4.0)"=>4.3,"(4.3, 3.3),(3.0, 3.0)"=>4.3))

4.3

In [53]:
#reset costs every 2 periods
function update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = copy(current_cs)
    
    #update costs
    for i=1:n
        next_ci = collect(next_cs[i])
        next_ci[1] =current_cs[i][1]
        if mod(t,2) == 0
            next_ci[1] = cs[rand(1:end)]
        end

        for s=0:(lag-1)
            next_ci[end-s] = current_cs[i][end-s-1]
        end
        next_cs[i] = Tuple(next_ci)
    end
    return next_cs
end


function update_public_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = copy(current_as)
    for i=1:n
        next_asi = collect(next_as[i])
        for j=0:(n-1)
            next_asi[mod(j+i-1,n)+1] = m(current_as[i],current_cs[i])
        end
        next_as[i] = Tuple(next_asi)
    end
    return next_as
end


function update_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = update_public_info(m,current_as,current_cs,cs,n,lag,t)
    return next_as, next_cs, t+1
end

update_info(m0, [(4.3, 4.3),(4.3,4.3)],[(4.,4.),(4.,4.)],cs,n,lag,2)

(Tuple{Float64,Float64}[(4.3, 4.3), (4.3, 4.3)], Tuple{Float64,Float64}[(4.0, 4.0), (4.0, 4.0)], 3)

In [55]:
BETA = .9
W0 = init_Ws(as,cs,n,lag,[10,11,12])

function update_ws(m,current_as,current_cs, n, beta, Ws)
    

    profit = zeros(n)
    policies = zeros(n)
    
    for i=1:n
        policies[i] = m( current_as[i], current_cs[i] )
    end
    println(policies)
    for i=1:n
        share = 1 - exp(policies[i]) / ( 1+sum(exp.(policies) ))
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        profit[i] = share*(policies[i] - current_cs[i][1]) + beta*Ws["$(state),$(current_cs[i])"][1]
    end
    println(profit)
    
    for i=1:n
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        weight = 1/( Ws["$(state),$(current_cs[i])"][2] +1 )
        println("weight $(i): ", weight, " ",  Ws["$(state),$(current_cs[i])"][2])
        println( Ws["$(state),$(current_cs[i])"][2] )
        new_w = weight*profit[i] + (1-weight)*Ws["$(state),$(current_cs[i])"][1]
        new_counter = Ws["$(state),$(current_cs[i])"][2] +1
        Ws["$(state),$(current_cs[i])"]= (new_w,new_counter)
    end
    
    return Ws
end

competition(x,y) = 3.3

update_ws( m0,[(3.3, 3.3),(3.3,3.3)],[(3.,3.),(3.,3.)], n, 0, W0 )

[4.3, 4.3]
[0.65438, 0.65438]
weight 1: 1.0 0
0
weight 2: 0.5 1
1


Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (0.65438, 2)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (11, 0

In [33]:
as = [3.3, 3.7 , 4.0, 4.3] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(4., 4.),(4.,4.)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = .9

for i=1:3000
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(4.3, 4.3, 4.3),(4.0, 4.0)(1.588091790067285, 2181)
(4.0, 3.7, 3.7),(3.0, 3.0)(3.712858571858168, 765)
(4.3, 3.7, 3.7),(4.0, 3.0)(1.7162379008502453, 748)
(4.0, 4.3, 4.3),(3.0, 4.0)(3.746368919673621, 750)
(3.7, 4.0, 4.0),(3.0, 3.0)(3.1265488014290583, 1515)
3.3: 8
3.7: 1521
4.0: 1527
4.3: 2944


In [34]:
as = [3.3, 3.7 , 4.0, 4.3] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(4., 4.),(4.,4.)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = .5

for i=1:3000
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(4.3, 4.3, 4.3),(4.0, 4.0)(0.326888593056219, 2178)
(4.3, 4.3, 4.3),(3.0, 4.0)(1.3187380723941386, 725)
(4.3, 4.3, 4.3),(4.0, 3.0)(0.33147053518874464, 715)
(4.3, 4.3, 4.3),(3.0, 3.0)(1.3095067818919273, 2214)
3.3: 17
3.7: 31
4.0: 64
4.3: 5888


In [32]:
as = [3.3, 4.3] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[5,6,7])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(4.3, 4.3),(4.3,4.3)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = 0

for i=1:3000
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] >= 1
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(4.3, 3.3, 3.3),(3.0, 4.0)(0.3589584263561248, 1)
(3.3, 4.3, 4.3),(4.0, 4.0)(-0.5135900817216599, 1)
(4.3, 4.3, 4.3),(4.0, 4.0)(0.150951708030386, 2308)
(4.3, 4.3, 4.3),(3.0, 3.0)(0.6541080387938105, 2172)
(3.3, 3.3, 3.3),(4.0, 3.0)(-0.3563376774854025, 2)
(3.3, 4.3, 4.3),(3.0, 3.0)(0.22011003502356832, 1)
(4.3, 3.3, 3.3),(3.0, 3.0)(0.3589584263561248, 1)
(4.3, 4.3, 4.3),(4.0, 3.0)(0.15101078448147223, 755)
(3.3, 4.3, 4.3),(3.0, 4.0)(0.22011003502356832, 1)
(3.3, 4.3, 4.3),(4.0, 3.0)(-0.5135900817216599, 1)
(4.3, 4.3, 4.3),(3.0, 4.0)(0.6543800660863804, 755)
(3.3, 3.3, 3.3),(4.0, 4.0)(-0.5135900817216599, 1)
(3.3, 3.3, 3.3),(3.0, 3.0)(0.22011003502356832, 1)
3.3: 8
4.3: 5992
